In [ ]:
library(rtweet)
library(ggmap)

In [ ]:
token <- create_token(app="TweetsFluCorrelation",
                      "", 
                      "",
                     access_token="",
                     access_secret="")
register_google(key = "")

In [ ]:
tweets2 <- search_tweets(
  "#flu OR flu OR #fluseason OR flu season OR #flushot OR flu shot OR #tamiflu OR tamiflu OR #gripe OR gripe", token=token, geocode = lookup_coords("usa"), 
n = 20000, include_rts=FALSE, retryonratelimit = TRUE)

#write_as_csv(tweets1, "tweets_mult_keyword2.csv")

In [ ]:
# new york city
tweets1 <- search_tweets(
  "#flu OR flu OR #fluseason OR flu season OR #flushot OR flu shot OR #tamiflu OR tamiflu OR #gripe OR gripe", token=token, geocode = lookup_coords("NYC"), 
n = 2000, include_rts=FALSE, retryonratelimit = TRUE)

#write_as_csv(tweets1, "tweets_mult_keyword2.csv")

In [ ]:
tweets1$location <- "NYC"

In [ ]:
tweets_dc <- search_tweets(
  "#flu OR flu OR #fluseason OR flu season OR #flushot OR flu shot OR #tamiflu OR tamiflu OR #gripe OR gripe", 
    token=token, geocode = lookup_coords("District of Columbia"), 
n = 2000, include_rts=FALSE, retryonratelimit = TRUE)

In [ ]:
tweets_dc$location <- "DC"

In [ ]:
tweets_pr <- search_tweets(
  "flu OR #flu OR #fluseason", 
    token=token, geocode = lookup_coords("Puerto Rico"), 
n = 2000, include_rts=FALSE, retryonratelimit = TRUE)

In [ ]:
length(tweets_pr$location)

In [ ]:
tweets_pr$location <- "Puerto Rico"

In [ ]:
tweetsdf1 <- read.csv("tweets_mult_keyword2.csv")
tweetsdf1 <- data.frame(tweetsdf1)

In [ ]:
length(tweetsdf1$location)
length(tweets1$location)
length(tweets_dc$location)
length(tweets_pr$location)

In [ ]:
tweetsdf1 <- rbind(tweetsdf1, tweets1)
tweetsdf1 <- rbind(tweetsdf1, tweets_dc)
tweetsdf1 <- rbind(tweetsdf1, tweets_pr)


In [ ]:
write_as_csv(tweetsdf1, "tweets_mult_keywordNEW.csv")

In [1]:
# you can start here to run 
library(rtweet)
library(ggmap)

Warning message:
“package ‘ggmap’ was built under R version 3.5.2”Loading required package: ggplot2
Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.
Please cite ggmap if you use it! See citation("ggmap") for details.


In [2]:
tweetsdf1<- read_twitter_csv("tweets_mult_keywordNEW.csv")

In [3]:
length(tweetsdf1$location)

[1] 29586

In [4]:
# getting states data frame
states <- read.csv(url("https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv"))
df <- as.data.frame(states)
head(states)


State,Abbreviation
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA
Colorado,CO


In [5]:
temp <- data.frame("State"=c("Puerto Rico", "New York City"),"Abbreviation"=c("PR","NYC"))

df1 <- rbind(df, temp)

In [6]:
#length(tweetsdf1$screen_name)
length(df1$State)

[1] 53

In [7]:
# need to get tweets with 2 different keywords, one for flu, another for flu season
hashflu <- tweetsdf1[grep("#flu",tweetsdf1$text, value=FALSE),]
fluseason <- tweetsdf1[grep("flu season", tweetsdf1$text, value=FALSE),]
length(hashflu$text) 
length(fluseason$text) 

[1] 1041

[1] 1156

In [8]:
get_num_state <- function(tweetsdf1){
    library("tidyverse")
    library("rlist")
    library(rlist)
# setting up tallies for each state starting at 0 obviously
    num_tweets <- rep(0, each=53)
    place_full_ab <- c()
    place_full_st <- c()
    loc_ab <- c()
    loc_st <- c()
    quoted_loc_ab <- c()
    quoted_loc_st <- c()

    for (i in 1:length(df1$State))
    {
        # check location
        loc_ab <- grep(df1$Abbreviation[i],  tweetsdf1$location, perl=TRUE, value=FALSE)
        loc_st <- grep(df1$State[i],  tweetsdf1$location, perl=TRUE, value=FALSE)
        # check place_full_name
        place_full_ab <- grep(df1$Abbreviation[i],  tweetsdf1$place_full_name, perl=TRUE, value=FALSE)
        place_full_st <- grep(df1$State[i], tweetsdf1$place_full_name, perl=TRUE, value=FALSE)
        # check quoted_location
        quoted_loc_ab <- grep(df1$Abbreviation[i], tweetsdf1$quoted_location, perl=TRUE, value=FALSE)
        quoted_loc_st <- grep(df1$State[i], tweetsdf1$quoted_location, perl=TRUE, value=FALSE)
        total <- list.append(place_full_ab, place_full_st)
        total <- list.append(total, loc_ab)
        total <- list.append(total, loc_st)
        total <- list.append(total, quoted_loc_ab)
        total <- list.append(total, quoted_loc_st)
        total <- !duplicated(total)
        #num_tweets[!duplicated(total)] <- num_tweets[!duplicated(total)] + 1
        num_tweets[i] <- num_tweets[i] + length(total)
    }
    return(num_tweets)
}



In [9]:
df1$MULT.KEYWORDS <-  get_num_state(tweetsdf1)
df1$KEYWORD.HASHFLU <- get_num_state(hashflu)
df1$KEYWORD.FLUSEASON <- get_num_state(fluseason)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  2.0.1       ✔ purrr   0.3.0  
✔ tidyr   0.8.2       ✔ dplyr   0.8.0.1
✔ readr   1.3.1       ✔ stringr 1.4.0  
✔ tibble  2.0.1       ✔ forcats 0.4.0  
Warning message:
“package ‘tibble’ was built under R version 3.5.2”Warning message:
“package ‘purrr’ was built under R version 3.5.2”Warning message:
“package ‘dplyr’ was built under R version 3.5.2”Warning message:
“package ‘stringr’ was built under R version 3.5.2”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks rtweet::flatten()
✖ dplyr::lag()     masks stats::lag()


In [10]:
head(df1)


State,Abbreviation,MULT.KEYWORDS,KEYWORD.HASHFLU,KEYWORD.FLUSEASON
Alabama,AL,457,15,16
Alaska,AK,17,1,1
Arizona,AZ,362,15,13
Arkansas,AR,267,11,6
California,CA,2297,72,86
Colorado,CO,333,10,24


In [11]:
# now we just plot them

In [12]:
# gotta get the ranges
# max - min to get the total amount
# piece divide up total by 10
# (min to (min + piece)), ((min + piece+1) to min + 2*piece), ((min+2*piece+1) to (min+3*piece))
get_ranges <- function(COLUMN)
{
    min_mult <- min(COLUMN)
    max_mult <- max(COLUMN)

    rng <- ((max_mult - min_mult) / 10)
    rng
    ranges_mult <- c(min_mult+rng, min_mult+2*rng, min_mult+3*rng, min_mult+4*rng, min_mult+5*rng,
           min_mult+6*rng, min_mult+7*rng, min_mult+8*rng, min_mult+9*rng, max_mult)
    mult_levels <- c()
    ranges_mult
    for (x in COLUMN)
    {
        for (i in 1:10){
            if (as.numeric(x) <= ranges_mult[i]){
                mult_levels <- c(mult_levels, i)
                break
            }
        }
    }
    return(mult_levels)
}

In [13]:
df1$MULT.LEVELS = get_ranges(df1$MULT.KEYWORDS)
df1$HASHFLU.LEVELS = get_ranges(df1$KEYWORD.HASHFLU)
df1$FLUSEASON.LEVELS = get_ranges(df1$KEYWORD.FLUSEASON)

In [14]:
head(df1)

State,Abbreviation,MULT.KEYWORDS,KEYWORD.HASHFLU,KEYWORD.FLUSEASON,MULT.LEVELS,HASHFLU.LEVELS,FLUSEASON.LEVELS
Alabama,AL,457,15,16,2,2,2
Alaska,AK,17,1,1,1,1,1
Arizona,AZ,362,15,13,2,2,2
Arkansas,AR,267,11,6,1,2,1
California,CA,2297,72,86,9,9,9
Colorado,CO,333,10,24,2,2,3


In [15]:
library(ggplot2)

In [16]:
getwd()

[1] "/Users/JohnnyD/Documents/CSE487/Lab1EDApart3"

In [17]:
# get all 50 states plus DC
df1$state_name = df1$State
df1$State = NULL

data <- read.csv("StateData.csv")
sts <- data.frame(data)
sts <- fortify(sts)
sts1 <- sts %>% left_join(df1, by = "state_name")

head(sts1)

Warning message:
“Column `state_name` joining factors with different levels, coercing to character vector”

X,long,lat,order,group,state_name,LEVEL,group2,Abbreviation,MULT.KEYWORDS,KEYWORD.HASHFLU,KEYWORD.FLUSEASON,MULT.LEVELS,HASHFLU.LEVELS,FLUSEASON.LEVELS
1,-88.47323,31.89386,1,1.1,Alabama,10,1,AL,457,15,16,2,2,2
2,-88.46888,31.93026,2,1.1,Alabama,10,1,AL,457,15,16,2,2,2
3,-88.46866,31.93317,3,1.1,Alabama,10,1,AL,457,15,16,2,2,2
4,-88.45504,32.03972,4,1.1,Alabama,10,1,AL,457,15,16,2,2,2
5,-88.45496,32.04058,5,1.1,Alabama,10,1,AL,457,15,16,2,2,2
6,-88.45342,32.05305,6,1.1,Alabama,10,1,AL,457,15,16,2,2,2


In [18]:
sts1$group <- sts1$group2
sts1$group2 <- NULL

In [19]:
# getting weird polygon issues with alaska, so gonna remove some coordinates
#sts1<- sts1[-seq(10000,11640,1),]

In [20]:
head(sts1)

X,long,lat,order,group,state_name,LEVEL,Abbreviation,MULT.KEYWORDS,KEYWORD.HASHFLU,KEYWORD.FLUSEASON,MULT.LEVELS,HASHFLU.LEVELS,FLUSEASON.LEVELS
1,-88.47323,31.89386,1,1,Alabama,10,AL,457,15,16,2,2,2
2,-88.46888,31.93026,2,1,Alabama,10,AL,457,15,16,2,2,2
3,-88.46866,31.93317,3,1,Alabama,10,AL,457,15,16,2,2,2
4,-88.45504,32.03972,4,1,Alabama,10,AL,457,15,16,2,2,2
5,-88.45496,32.04058,5,1,Alabama,10,AL,457,15,16,2,2,2
6,-88.45342,32.05305,6,1,Alabama,10,AL,457,15,16,2,2,2


In [21]:

df_mult_key <- sts1[,c("state_name","MULT.LEVELS","long","lat","group")]
df_mult_key$LEVEL <- df_mult_key$MULT.LEVELS
df_mult_key$MULT.LEVELS <- NULL
df_flu_key <- sts1[,c("state_name","HASHFLU.LEVELS","long","lat","group")]
df_flu_key$LEVEL <- df_flu_key$HASHFLU.LEVELS
df_flu_key$HASHFLU.LEVELS <- NULL
df_fluseason_key <- sts1[,c("state_name","FLUSEASON.LEVELS","long","lat","group")]
df_fluseason_key$LEVEL <- df_fluseason_key$FLUSEASON.LEVELS
df_fluseason_key$FLUSEASON.LEVELS <- NULL

In [22]:
#head(df_mult_key)
#head(df_flu_key)
#head(df_fluseason_key)
#head(df_mult_key)

In [23]:
# and here we do the extremely tedious task of putting lines and labels to the legend items
# heavily taken from stackoverflow
# create data to plot the legend
# x and y to create a vertical row of points
# all levels of the variable to be represented in the legend (here z)
library(data.table)
d <- data.table(x = 1, y = 1:10, z = factor(1:10))
colors <- c("10"="#CC0000","9"="#FF4500","8"="#FF8C00", "7"="#FFA500","6"="#FFD700", "5"="#E6FF00","4"="#ABFA33","3"="#97FA33","2"="#3EFC04","1"="#0BE404")
# cut z into groups which should be displayed as text in legend
d[ , grp := cut(as.numeric(z), breaks = c(0, 2, 5, 7, 11),
                labels = c("   Minimal", "Low", "      Moderate", "High"))]

# calculate the start, end and mid points of each group
# used for vertical segments
d2 <- d[ , .(x = 1, y = min(y), yend = max(y), ymid = mean(y)), by = grp]

# end points of segments in long format, used for horizontal 'ticks' on the segments  
d3 <- data.table(x = 1, y = unlist(d2[ , .(y, yend)]))

# offset
v <- 0.5
# plot the 'legend'
p2 <- ggplot(mapping = aes(x = x-1, y = y)) +
  geom_point(data = d, aes(color = z), size = 5) +
  geom_segment(data = d2,
               aes(x = x + v-1, xend = x + v-1, yend = yend)) +
  geom_segment(data = d3,
               aes(x = x + v-1, xend = x + (v - 0.1)-1, yend = y)) +
  geom_text(data = d2, aes(x = x + v - 0.4, y = ymid, label = grp)) +
  scale_color_manual(values = colors, guide = FALSE) +
  scale_x_continuous(limits = c(0, 4)) +
  theme(axis.text.x = element_blank(), 
        axis.text.y = element_blank(), 
        axis.ticks = element_blank(),
        axis.title.y=element_blank(),
        axis.title.x=element_blank(),
        axis.line=element_blank())+
coord_fixed(ratio = 0.4)+
  theme_void()



Warning message:
“package ‘data.table’ was built under R version 3.5.2”
Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [24]:
#Ok now we plot

# has all 50 states plus DC
plot_heatmap <- function (df,str)
{

    attach(df)

    
# plotting single keyword 
    


    g <- ggplot(df ) +
      geom_polygon(aes(long, lat, group = group, fill=factor(LEVEL)),color = "gray", size = .05) +
      scale_fill_manual(breaks=LEVEL, values=colors) + 
      annotate("text", label = "Alaska", x =-122.794213 , y = 30.569676, size = 3, colour = "black")+
      annotate("text", label = "Hawaii", x =-108.794213 , y = 29.569676, size = 3, colour = "black")+
      annotate("text", label = "Puerto Rico", x =-88.794213 , y = 25.569676, size = 3, colour = "black")+
      annotate("text", label = "District of Columbia", x =-70.794213 , y = 34.569676, size = 3, colour = "black")+
      annotate("text", label = "New York City", x =-68.794213 , y = 38.569676, size = 3, colour = "black")+
  #,breaks=c("10","9", "8","7",  "6","5", "4",  "3", "2","1"),
  #  labels=c("","","","","","","","","",""))+
      coord_map(projection = "mercator") +
      ggtitle(paste(str,"\nFeb 16th to 26th, 2019",sep=""))+
      theme(plot.title = element_text(face = "bold",hjust=.5,size=11), 
        legend.position = "none",
        axis.text.x = element_blank(), axis.text.y = element_blank(), axis.ticks = element_blank(),
        axis.title.y=element_blank(),axis.title.x=element_blank(),
        axis.line=element_blank()) 
    detach(df)
    return (g)
}

In [25]:
#head(df_mult_key$LEVEL)

In [26]:
#plot_heatmap(df_mult_key,"Multiple Keywords Related to Flu, ")

In [38]:
library(cowplot)
pg <- plot_grid(plot_heatmap(df_mult_key,"Multiple Keywords Related to Flu, "),
    plot_grid(NULL, p2, NULL, nrow = 3, rel_heights = c(1, 1.5, 1)),
    rel_widths = c(2, 1)) +draw_label("Tweets per State", .65, 0.75, hjust = 0, vjust = 0,size=10)




In [40]:
ggsave("HeatMapMultKeys.png",pg,width=7,height=4)

In [29]:
pg2 <- plot_grid(plot_heatmap(df_flu_key, "Tweet Keyword '#flu'"),
    plot_grid(NULL, p2, NULL, nrow = 3, rel_heights = c(1, 1.5, 1)),
    rel_widths = c(2, 1)) +draw_label("Tweets per State", .65, 0.75, hjust = 0, vjust = 0,size=10)

In [34]:
ggsave("HeatMapHashFluKey.png",pg2,height=4)

Saving 7 x 4 in image


In [31]:
pg3 <- plot_grid(plot_heatmap(df_fluseason_key, "Tweet Keyword 'flu season'"),
    plot_grid(NULL, p2, NULL, nrow = 3, rel_heights = c(1, 1.5, 1)),
    rel_widths = c(2, 1)) +draw_label("Tweets per State", .65, 0.75, hjust = 0, vjust = 0,size=10)

In [35]:
ggsave("HeatMapFluSeasonKey.png",pg3,height=4)

Saving 7 x 4 in image
